In [1]:
from __future__ import division
from __future__ import print_function
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, auc
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

import re
from HTMLParser import HTMLParser
import datetime
import cPickle as pickle
pd.options.mode.chained_assignment = None  # default='warn'

from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

In [ ]:
# Files
all_file = '/Users/lekha/galvanize/capstone/projectRiley/data/withindgroup/all_withindgroup.txt'
tech_file = '/Users/lekha/galvanize/capstone/projectRiley/data/withindgroup/tech_withind.txt'
hadoop_all = '/Users/lekha/galvanize/capstone/projectRiley/data/withindgroup/all_hadoop.txt'
hadoop_tech = '/Users/lekha/galvanize/capstone/projectRiley/data/withindgroup/tech_hadoop.txt'

In [ ]:
dfall1 = pd.read_csv(all_file, sep="|")
dftech1 = pd.read_csv(tech_file, sep="|")
dfh = pd.read_csv(hadoop_all, sep="|")
dfht = pd.read_csv(hadoop_tech, sep="|")

In [2]:
dftech_all = pd.concat([dftech1, pd.read_csv(hadoop_tech, sep="|")], axis=0)

NameError: name 'dftech1' is not defined

In [3]:
df_all2 = pd.concat([dfall1, pd.read_csv(hadoop_all, sep="|")], axis=0)

NameError: name 'dfall1' is not defined

In [ ]:
df_all2.to_csv('/Users/lekha/galvanize/capstone/projectRiley/data/withindgroup/all2.txt', sep="|", index=False)
dftech_all.to_csv('/Users/lekha/galvanize/capstone/projectRiley/data/withindgroup/tech_all2.txt', sep="|", index=False)

In [ ]:
df = df_all2.copy()

In [ ]:
df['first_name'] = df['first_name'].apply(lambda x:'andrew' if x=='anew' else x)
df.loc[df.first_name.isin(['andrew']), 'gender'] = 'male'

# Modeling - Original Data

In [ ]:
# Functions
def missing(df):
    if df.summary == 'missing' or df.num_tokens == 0:
        return 1
    else:
        return 0    

    
def lenx(mystr):
    return len(mystr.split())


def avgchrs(mytokens):
    tw = len(mytokens)    
    num_chars = 0
    for word in mytokens:
        num_chars += len(word)        
    return num_chars/tw


def remove_digits(mystr):
    '''
    INPUT: list of tokens 
    OUTPUT: list of tokens with digits removed
    '''
    return [word for word in mystr if not word.isdigit()]


stemmer = SnowballStemmer("english")
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed


def tokenize(text):
    tokens = word_tokenize(text)
    row = remove_digits(tokens)
    stems = stem_tokens(row, stemmer)
    return stems


def tokenize_no_stem(text):
    tokens = word_tokenize(text)
    row = remove_digits(tokens)
    stems = stem_tokens(row, stemmer)
    return tokens 


In [ ]:
def preprocess_df(df):
    # Feature Engineering before running the prediction code
    df['class'] = np.ones(len(df))
    df['class'] = df['gender_forced'].apply(lambda x: 0 if x == 'female' else 1)

    df['summ_tokens'] = df['summary'].apply(lambda x: nltk.word_tokenize(str(x)))
    df['num_tokens'] = df['summ_tokens'].apply(lambda x: len(x))

    # Add feature for missing summary
    df['summ_missing'] = df.apply(missing, axis = 1)

    # Only include rows with summaries
    df = df[df['summ_missing'] == 0]

    # Some Nan rows refuse to go without this
    df = df[pd.notnull(df['summary'])]

    print ("Length of DF after removing rows with missing Summaries:\n")
    print (len(df))

    df['avg_len'] = df['summ_tokens'].apply(lambda x: avgchrs(x))

    # lexical diversity = number of unique tokens / total number of tokens
    df['lex_diversity'] = df['summ_tokens'].apply(lambda x: len(set(x))/len(x))
    
    return df

In [ ]:
def rf_predict(df):
# Train-test split

    X_train, X_test, y_train, y_test = train_test_split(df['summary'], df['class'], test_size=0.3, random_state=0)
    vectorizer = CountVectorizer(analyzer = 'word', tokenizer = tokenize, ngram_range=(1,4), stop_words = stopwords, max_df = 0.7, min_df = 5, max_features = 5000)
    train_fit = vectorizer.fit_transform(X_train)
    train_fit = train_fit.toarray()
    feature_names = vectorizer.get_feature_names()

    print ("Training the random forest...")

    # Initialize a Random Forest classifier with 100 trees
    forest = RandomForestClassifier(n_estimators = 100) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the response variable
    #
    # This may take a few minutes to run
    forest = forest.fit(train_fit, y_train)

    # Testing

    # Get a bag of words for the test set, and convert to a numpy array
    #test_data_features = vectorizer.transform(X_test)
    test_features = vectorizer.transform(X_test)
    test_features = test_features.toarray()

    # Use the random forest to make sentiment label predictions
    yhat = forest.predict(test_features)
    probX = forest.predict_proba(test_features)

    print ("Precision Score: {0}".format(precision_score(y_test, yhat)))
    print ("Recall Score: {0}".format(recall_score(y_test, yhat)))
    print ("AUC Score: {0}".format(roc_auc_score(y_test, yhat)))
    print ("Model Score:{0}".format(forest.score(test_features, y_test)))
    
    fpr, tpr, thresholds = roc_curve(y_test, probX[:,1])    
    
    # Most important features
    num = 50
    imp = forest.feature_importances_
    feature_names = np.array(feature_names)
    features = feature_names[np.argsort(imp)[-num:]]
    weights = imp[np.argsort(imp)[-num:]]
    feats_with_vals = zip(features, weights)
    print ("Top 50 Features\n")
    for x in feats_with_vals:
        print (x)

        
    ## Naive Bayes Classifier
    nb_model = MultinomialNB()
    fitted_data = nb_model.fit(train_fit, y_train)
    yhat_nb = nb_model.predict(test_features)
    prob_nb = nb_model.predict_proba(test_features)
    
    print (nb_model)
    print ("Precision Score: {0}".format(precision_score(y_test, yhat_nb)))
    print ("Recall Score: {0}".format(recall_score(y_test, yhat_nb)))
    print ("AUC Score: {0}".format(roc_auc_score(y_test, yhat_nb)))
    print ("Model Score:{0}".format(nb_model.score(test_features, y_test)))

    fpr_nb, tpr_nb, thresholds_nb = roc_curve(y_test, prob_nb[:,1])
    
    ## Logistic Regression
    lr_model = LogisticRegression()
    fitted_data = lr_model.fit(train_fit, y_train)
    yhat_lr = lr_model.predict(test_features)
    prob_lr = lr_model.predict_proba(test_features)
    
    print (lr_model)
    print ("Precision Score: {0}".format(precision_score(y_test, yhat_lr)))
    print ("Recall Score: {0}".format(recall_score(y_test, yhat_lr)))
    print ("AUC Score: {0}".format(roc_auc_score(y_test, yhat_lr)))
    print ("Model Score:{0}".format(lr_model.score(test_features, y_test)))

    fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, prob_lr[:,1])
        
    
    # ROC Curve
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)

    plt.plot(np.array(range(101))/100, np.array(range(101))/100, '--', color='black')
    plt.plot(fpr, tpr, label='rf-tfidf')
    plt.plot(fpr_nb, tpr_nb, label='Naive Bayes')
    plt.plot(fpr_lr, tpr_lr, label='Logistic')
    plt.ylabel("True Positive Rate ")
    plt.xlabel("False Positive Rate")
    plt.title("ROC plot")
    plt.legend(loc='lower right')

    plt.grid()
    plt.show()

In [ ]:
# Model 1
rf_predict(df)

In [ ]:
dftech1 = preprocess_df(dftech1)
rf_predict(dftech1)

In [ ]:
dfh = preprocess_df(dfh)
rf_predict(dfh)

In [ ]:
dfht = preprocess_df(dfht)
rf_predict(dfht)

In [ ]:
df_all2 = preprocess_df(df_all2)
rf_predict(df_all2)

In [ ]:
dftech_all = preprocess_df(dftech_all)
rf_predict(dftech_all)

In [ ]:
# with 3 grams

df_all2 = preprocess_df(df_all2)
rf_predict(df_all2)

In [ ]:
# with 3 grams and Adding Naive Bayes and Logistic Regression

df_all2 = preprocess_df(df_all2)
rf_predict(df_all2)

In [ ]:
dftech_all = preprocess_df(dftech_all)
rf_predict(dftech_all)

In [ ]:


data = {'Weights':weights,
        'Labels':labels}



featdf = pd.DataFrame(data, columns = ['Weights', 'Features'])

featdf.plot(kind='bar', x=featdf['Weights'], fontsize=14, figsize=(8, 5), legend=False)


# df = pd.DataFrame({'score':np.random.randn(6),
#                    'person':[x*3 for x in list('ABCDEF')]})

# ax = plt.subplot(111)
# df.score.plot(ax=ax, kind='barh', color=list('rgbkym'), title='ranking')
# ax.axis('off')
# for i, x in enumerate(df.person):
#     ax.text(0, i + .5, x, ha='right', fontsize='large')



In [ ]:
## Printing out the Random Forest Tree

X_train, X_test, y_train, y_test = train_test_split(df['summary'], df['class'], test_size=0.3, random_state=0)
vectorizer = CountVectorizer(analyzer = 'word', tokenizer = tokenize, ngram_range=(1,4), stop_words = stopwords, max_df = 0.7, min_df = 5, max_features = 5000)
train_fit = vectorizer.fit_transform(X_train)
train_fit = train_fit.toarray()
feature_names = vectorizer.get_feature_names()

print ("Training the random forest...")

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(train_fit, y_train)

# Testing

# Get a bag of words for the test set, and convert to a numpy array
#test_data_features = vectorizer.transform(X_test)
test_features = vectorizer.transform(X_test)
test_features = test_features.toarray()

# Use the random forest to make sentiment label predictions
yhat = forest.predict(test_features)
probX = forest.predict_proba(test_features)

print ("Precision Score: {0}".format(precision_score(y_test, yhat)))
print ("Recall Score: {0}".format(recall_score(y_test, yhat)))
print ("AUC Score: {0}".format(roc_auc_score(y_test, yhat)))
print ("Model Score:{0}".format(forest.score(test_features, y_test)))

fpr, tpr, thresholds = roc_curve(y_test, probX[:,1])    

# Most important features
num = 50
imp = forest.feature_importances_
feature_names = np.array(feature_names)
features = feature_names[np.argsort(imp)[-num:]]
weights = imp[np.argsort(imp)[-num:]]
feats_with_vals = zip(features, weights)
print ("Top 50 Features\n")
for x in feats_with_vals:
    print (x)


In [ ]:
labels

In [ ]:
wts